In [1]:
from sqlglot import parse_one, exp
from sqlglot.dialects.tsql import TSQL
import pypyodbc as odbc
odbc.lowercase = False
import configparser
import copy
from collections import defaultdict
from collections import OrderedDict 
import pandas as pd
import pypyodbc as odbc
import configparser
odbc.lowercase = False


In [9]:


"""
Trial for extracting eveything from the subqueries and stockpiling them in the SELECT statement of the main query, first replacing
the column names and then the table aliases to create an easier lineage extract possibility.
"""

#This line to be repalced by the code to extract the tables and columns from a specific database (Erwin's database extractor tool)
hardcode_dict = [{"MA_NorthWindDB.Order Details": ["OrderID", "ProductID", "UnitPrice", "Quantity", "Discount"]}, {"dbo.Order Details": ["Product", "Sale", "Prices"]} ]


query = """
INSERT INTO MA_NorthWindDB.dbo.Order_Details_1998_Extract(star1, star2, star3, star4, star5,orderid, NrOfProducts, avg_order_unitprice, max_order_discount, min_order_discount, total_quantity, product_quantity_year, 	perc_of_product_quantity_year)

SELECT
    a.*,
    orderid,
	c.NrOfProducts, 
	c.avg_order_unitprice,
	c.max_order_discount, 
	c.min_order_discount, 
	c.total_quantity,
	d.product_quantity_year,
   round((cast(a.quantity as numeric (10,2))/cast(d.product_quantity_year as numeric (10,2)))*100,1) as perc_of_product_quantity_year
FROM 
	MA_NorthWindDB.[Order Details] a INNER JOIN MA_NorthWindDB.dbo.Orders b
	on a.OrderID = b.OrderID INNER JOIN MA_NorthWindDB.dbo.[Products] q
	on a.ProductID = q.ProductID
	INNER JOIN 
		(	
			select 
				orderid,
             count(Products) as NrOfProducts,
				max(avg(unitprice)) as avg_order_unitprice,
				max(Discount) as max_order_discount,  
				min(Discount) as min_order_discount, 
				sum(quantity) as total_quantity
			from 
				dbo.[Order Details] ode
			group by 
				OrderID
		) c

	on a.OrderID = c.OrderID	
	INNER JOIN 
		(
			select 
				productID, 
				sum(avg(quantity)) as product_quantity_year
			from 
				dbo.[Order Details] ode 
			group by 
				productID
		) d
	on a.productID = d.productID
where 
	YEAR(b.OrderDate) = 1998;

 
"""

query = '''CREATE VIEW ORDER_RECONCILIATION AS
SELECT A.CustomerId, C.CustomerName,  COUNT( DISTINCT A.OrderId) TotalNBOrders, COUNT( DISTINCT A.InvoiceId) TotalNBInvoices,
       SUM(A.UnitPrice*A.Quantity)AS OrdersTotalValue,  SUM(A.UnitPriceI * A.QuantityI) AS InvoicesTotalValue,
	   ABS(SUM(A.UnitPrice * A.Quantity) -  SUM(A.UnitPriceI*A.QuantityI)) AS AbsoluteValueDifference
FROM 
(
	SELECT O.CustomerID, O.OrderId, NULL AS InvoiceID, OL.UnitPrice, OL.Quantity, 0 AS UnitPriceI, 0 AS QuantityI, OL.OrderLineID, NULL AS InvoiceLineID 
	FROM Sales.Orders As O, Sales.OrderLines AS OL
	WHERE O.OrderId = OL.OrderID AND EXISTS
	(	SELECT II.OrderId
		FROM Sales.Invoices AS II
		WHERE II.OrderID = O.OrderID
	)
	UNION
	SELECT I.CustomerID, NULL AS OrderId, I.InvoiceID, 0 AS UnitPriceO, 0 AS QuantityO, IL.UnitPrice, IL.Quantity, NULL AS OrderLineID, InvoiceLineID
	FROM Sales.Invoices AS I, Sales.InvoiceLines AS IL
	WHERE I.InvoiceID = IL.InvoiceID
) AS A, Sales.Customers As C
WHERE A.CustomerID = C.CustomerID
GROUP BY A.CustomerID, C.CustomerName
ORDER BY AbsoluteValueDifference DESC, TotalNBOrders, CustomerName;
'''

ast = parse_one(query, read="tsql")
trial1 = repr(ast)

In [25]:
subqueries = list(ast.find_all(exp.Subquery))   
subqueries

selects = list(ast.find_all(exp.Select))   
selects

[Select(
   expressions=[
     Column(
       this=Identifier(this=CustomerId, quoted=False),
       table=Identifier(this=A, quoted=False)),
     Column(
       this=Identifier(this=CustomerName, quoted=False),
       table=Identifier(this=C, quoted=False)),
     Alias(
       this=Count(
         this=Distinct(
           expressions=[
             Column(
               this=Identifier(this=OrderId, quoted=False),
               table=Identifier(this=A, quoted=False))])),
       alias=Identifier(this=TotalNBOrders, quoted=False)),
     Alias(
       this=Count(
         this=Distinct(
           expressions=[
             Column(
               this=Identifier(this=InvoiceId, quoted=False),
               table=Identifier(this=A, quoted=False))])),
       alias=Identifier(this=TotalNBInvoices, quoted=False)),
     Alias(
       this=Sum(
         this=Mul(
           this=Column(
             this=Identifier(this=UnitPrice, quoted=False),
             table=Identifier(this=A, quoted

In [10]:
# Find all table names which have an empty space in them and storing them without the " " for later use.

table_names = list(ast.find_all(exp.Table))
space_table = []
for element in table_names:
    if " " in element.name:
        space_table.append((element.name.replace(" ",""),element.name))

space_table = list(set(space_table)) # a list of tuples with table names paired (space removed original - original ) Eg. (OrderDetails, Order Details)
space_table

[]

In [21]:
def parse_table(table, table_alias_list, subquery=True):    
    #table = element.this.this
    #table_name = table.name
    #catalog =  table.catalog
    #db = table.db

    if subquery == False:
        table_alias =  table.alias.strip()
        table_name = table.name.strip()
        table_db = table.db.strip()
        table_catalog = table.catalog.strip()

    else:
        table_alias = table.alias.strip()
        source = table.this.args#["from"].strip()
        table_name= source.this.name.strip()
        table_catalog =  source.this.catalog.strip()
        table_db = source.this.db.strip()
        
 
    if " " in table_name:
        table_name = table_name.replace(" ", "")
        

    if table_catalog != "" and table_db != "":
        result = (table_catalog+"."+ table_db+"."+table_name, table_alias)

    elif table_db == "" and table_catalog == "":

        result = (table_name, table_alias)

    elif table_catalog == "": 
        result = (table_db+"."+table_name, table_alias)

    elif table_db == "":
        result = (table_catalog+"."+table_name, table_alias)
        

    table_alias_list.append(result)
    return result



In [24]:
table_alias = list(ast.find_all(exp.Table))


alias_table = []
for table in table_alias:
    parse_table(table, alias_table, False)

print(alias_table)

subqueries = list(ast.find_all(exp.Subquery))   
subquery_aliases = [] 
if len(subqueries) > 0:
    for subquery in subqueries:
            parse_table(subquery, alias_table, True)
                
#The object subquery_aliases is a list of tuples with the alias strcuture (database name + table name , alias of the subquery)

#alias_table = alias_table + subquery_aliases #Adding the table+alias tuples from the subqueries to the general alias_table object.
alias_table


[('ORDER_RECONCILIATION', ''), ('Sales.Customers', 'C'), ('Sales.Orders', 'O'), ('Sales.OrderLines', 'OL'), ('Sales.Invoices', 'I'), ('Sales.InvoiceLines', 'IL'), ('Sales.Invoices', 'II')]


In [13]:
# parse table name + table alias

# Extract the table names and their aliases, used to reconstruct a tuple with structure (database+schema+name, alias )

table_alias = list(ast.expression.find_all(exp.Table))
alias_table = []
for table in table_alias:
    parse_table(table, alias_table, False)

#The object alias_table contains all the (database+table, alias) tuple combinations for later use and replacement.

#Once again extracting the table names and aliases but this time for the tables mentioned in the subqueries of the sql script.
#The only drawback is that this only works when the subquery is only one level nested and only one table is mentioned in the From statement of the subquery.
#It pairs the table name with the alias of the whole subquery.
subqueries = list(ast.find_all(exp.Subquery))   
subquery_aliases = [] 
if len(subqueries) > 0:
    for subquery in subqueries:
            parse_table(subquery, alias_table, False)
                
#The object subquery_aliases is a list of tuples with the alias strcuture (database name + table name , alias of the subquery)

#alias_table = alias_table + subquery_aliases #Adding the table+alias tuples from the subqueries to the general alias_table object.
alias_table


AttributeError: 'Subquery' object has no attribute 'db'

In [20]:
# tables + columns

#Below code snippet iterates over the subqueries, collects the table name from the FROM statement and also paires it with the mentioned column objects.
#pseudo_tables object is a list of dictionaries where the key is the table name from the FROM statement and the values are all the columns
#mentioned in the subquery. Basicly creating artificial tables from the subqueries and storing them in a dictionary format. 
pseudo_tables = []
for element in subqueries:
    
    expressions = list(element.this.expressions)
    column_objects = [list(x.find_all(exp.Column)) for x in expressions]
    variables = []
    for l in column_objects:
        for o in l:
            variables.append(o.name)
            
    variables = list(set(variables))
    alias = element.alias
    source = element.this.args["from"]
    table= source.this.name
    catalog =  source.this.catalog
    db = source.this.db
    complete_table = ""
    if " " in table:
        table = table.replace(" ", "")
    if catalog =="" and db =="":
        complete_table = table
    elif catalog == "":    
        complete_table = db + "." + table
    elif db == "":
        complete_table = catalog + "." + table
    else:
        complete_table = catalog + "." +  db +"." + table
    temp  = {}
    temp[complete_table] = variables
    pseudo_tables.append(temp)
    
pseudo_tables


KeyError: 'from'

In [ ]:
# column name + columns alias

#The below snippet iterates over the subqueries and extracts all the column names with their respective aliases. It does this by going over
#a tree branch and looking for values in the column objects that are strings. It can also identify if the column call is a star. If no alias was
#used the code returns "NoAlias" in place of the alias.
function_lines = []
column_subquery_alias=[]
name_subquery_columns=[]
for subs in subqueries:
    full_lines = []
    columns2 = subs.this.args["expressions"]
    for element in columns2:
        temp = element.this
        while isinstance(temp, str) ==False:
            if isinstance(temp, exp.Star):
                name_subquery_columns.append("*")
                break
            temp = temp.this

        if isinstance(temp, str):
            name_subquery_columns.append(temp)
   
    full_lines = []
    for element in columns2:
        temporary_info= []
        if isinstance(element, exp.Alias) ==True:
            alias = element.alias
        else:
            alias = "NoAlias"
        columns_mentioned = list(element.find_all(exp.Column))
        functions = list(element.find_all(exp.Func))
        full_lines.append(([x.key for x in functions], [y.name for y in columns_mentioned], alias))
    
    string_lines = []
    for element in full_lines:
        if element [0]!= []:
            functions_string = "(".join(element[0])+"("
        else:
            functions_string = ""
        columns_string = ",".join(element[1])
        lenght = len(element[0])
        functions_string = functions_string+ columns_string + lenght * ")"
        string_lines.append((functions_string, element[2]))
    function_lines.append(string_lines)

subquery_matched = []
for element in function_lines:
    for i in element:
        subquery_matched.append(i)


subquery_matched


[('orderid', 'NoAlias'),
 ('count(Products)', 'NrOfProducts'),
 ('max(avg(unitprice))', 'avg_order_unitprice'),
 ('max(Discount)', 'max_order_discount'),
 ('min(Discount)', 'min_order_discount'),
 ('sum(quantity)', 'total_quantity'),
 ('productID', 'NoAlias'),
 ('sum(avg(quantity))', 'product_quantity_year')]

In [ ]:
# translate sql dialect

#SQLGlot's special functions are transformer functions which iterate over each node and make changes on the way. These functions are designed
#for this specific code.

def transformer_column(node):
    """This first function loops over the main SELECT node and replaces column names that appear in the subqueries as aliases"""
    for element in subquery_matched:
        if isinstance(node, exp.Column) and node.name == element[1]:
            return parse_one(element[0])
    return node

def transformer_table(node):
    for element in alias_table:
        if isinstance(node, exp.Column) and node.table == element[1]:
            return parse_one(element[0] + "." + node.name)
    return node

def transformer_subquery(node):
    for element in pseudo_tables:
        for i,j in element.items():
            for l in j:
                if isinstance(node, exp.Column) and node.table =="" and node.name ==l:
                    return parse_one(i + "." + l)
    return node

transformed_tree = ast.transform(transformer_table).transform(transformer_column).transform(transformer_subquery)
transformed_trial =transformed_tree.sql(dialect = 'tsql')

syntax = repr(transformed_tree)
print(syntax)

Insert(
  this=Schema(
    this=Table(
      this=Identifier(this=Order_Details_1998_Extract, quoted=False),
      db=Identifier(this=dbo, quoted=False),
      catalog=Identifier(this=MA_NorthWindDB, quoted=False)),
    expressions=[
      Identifier(this=star1, quoted=False),
      Identifier(this=star2, quoted=False),
      Identifier(this=star3, quoted=False),
      Identifier(this=star4, quoted=False),
      Identifier(this=star5, quoted=False),
      Identifier(this=orderid, quoted=False),
      Identifier(this=NrOfProducts, quoted=False),
      Identifier(this=avg_order_unitprice, quoted=False),
      Identifier(this=max_order_discount, quoted=False),
      Identifier(this=min_order_discount, quoted=False),
      Identifier(this=total_quantity, quoted=False),
      Identifier(this=product_quantity_year, quoted=False),
      Identifier(this=perc_of_product_quantity_year, quoted=False)]),
  expression=Select(
    expressions=[
      Column(
        this=Star(),
        table=Identi